## Configuration

Update these settings to match your environment:

In [ ]:
# OIDC Configuration
OIDC_BASE_URL = "http://localhost:3000/oidc"  # Your OIDC server URL
CLIENT_ID = "default-client"                  # Your OAuth client ID
CLIENT_SECRET = "default-secret"              # Your OAuth client secret
SCOPE = "openid profile email"                # Requested scopes

print("✓ Configuration loaded")
print(f"  OIDC URL: {OIDC_BASE_URL}")
print(f"  Client ID: {CLIENT_ID}")

## Install Dependencies

Install required packages for HTTP requests:

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("https://exabyte-io.github.io/api-examples/mat3ra_api_examples-0.1.dev1+gbff9dd7da-py3-none-any.whl", deps=False)
    await micropip.install("mat3ra-utils")
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("api_examples")

## Authentication Function

This function implements the complete device code flow:

In [ ]:
from utils.auth import authenticate_device_flow

try:
    token_data = await authenticate_device_flow(
        oidc_base_url=OIDC_BASE_URL,
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
        scope=SCOPE,
    )
    
    print("\n📋 Token Information:")
    print(f"  Token Type: {token_data.get('token_type', 'N/A')}")
    print(f"  Expires In: {token_data.get('expires_in', 'N/A')} seconds")
    print(f"  Access Token (first 50 chars): {token_data['access_token'][:50]}...")
    if 'refresh_token' in token_data:
        print(f"  Refresh Token: Available")
    
except Exception as e:
    print(f"\n❌ Authentication failed: {e}")

## Using the Access Token

Now you can use the access token to make authenticated API requests:

In [ ]:
def call_api(endpoint, method="GET", data=None):
    """
    Make an authenticated API request.
    
    Args:
        endpoint: API endpoint (e.g., '/api/v1/users/me')
        method: HTTP method (GET, POST, PUT, DELETE)
        data: Request body for POST/PUT requests
    
    Returns:
        Response data
    """
    access_token = os.environ.get('OIDC_ACCESS_TOKEN')
    
    if not access_token:
        raise Exception("No access token found. Please authenticate first.")
    
    # Construct full URL
    base_url = OIDC_BASE_URL.replace('/oidc', '')  # Remove /oidc suffix
    url = f"{base_url}{endpoint}"
    
    # Make request
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
    }
    
    response = requests.request(
        method=method,
        url=url,
        headers=headers,
        json=data,
        timeout=30,
    )
    
    if response.status_code >= 400:
        raise Exception(f"API request failed ({response.status_code}): {response.text}")
    
    return response.json() if response.headers.get('content-type', '').startswith('application/json') else response.text


print("✓ API helper function defined")

### Example: Get Current User Info

In [ ]:
try:
    user_info = call_api('/api/v1/users/me')
    print("👤 User Information:")
    print(user_info)
except Exception as e:
    print(f"❌ Error: {e}")

### Example: List Projects

In [ ]:
try:
    projects = call_api('/api/v1/projects')
    print(f"📁 Projects ({len(projects)} total):")
    for project in projects[:5]:  # Show first 5
        print(f"  - {project.get('name', 'Unnamed')} (ID: {project.get('_id', 'N/A')})")
except Exception as e:
    print(f"❌ Error: {e}")